In [11]:
import requests
import psycopg2
from psycopg2 import sql
import os
from dotenv import load_dotenv
import pandas as pd
from Bitcoin_api import fetch_bitcoin_data
from new_scrappy import get_bitcoin_news

Python-dotenv could not parse statement starting at line 11


Fetched DataFrame:
             1. Open   2. High    3. Low  4. Close     5. Volume
Date                                                            
2024-06-02  67719.29  67764.08  67689.70  67734.14     69.098162
2024-06-01  67473.07  67837.33  67371.28  67719.29   1949.570081
2024-05-31  68338.58  69024.71  66584.47  67472.41  10690.858762
2024-05-30  67569.44  69536.89  67092.91  68338.58  11841.263819
2024-05-29  68321.99  68864.11  67081.66  67569.45   8970.020591
[Total Rows: 180]


In [17]:
from sqlalchemy import create_engine

In [18]:
print("Hello World!")

Hello World!


# BTC

## Connecting to DataBase and Creating Tables

In [8]:
# make sure to load dotenv
load_dotenv()

# Database connection paramete(s
Database_URL = os.getenv('DataBase_Url')

#connect to database
conn = psycopg2.connect(Database_URL)
cur = conn.cursor()

#create tables if they don't exist

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_Prices_Update(
    Date DATE PRIMARY KEY,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Volume FLOAT
    
)            
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_News_Update(
    id SERIAL PRIMARY KEY,
    date DATE,
    title VARCHAR(500)
       
)            
''')

conn.commit()

Python-dotenv could not parse statement starting at line 11


## Insertion of Bitcoin Prices_Update to Table

In [12]:
#function to insert Bitcoin_Prices
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_Prices_Update(Date, Open, High, Low, Close, Volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Date) DO NOTHING
                ''',(index, row['1. Open'], row['2. High'], row['3. Low'], row['4. Close'], row['5. Volume']))
        conn.commit()
        cursor.close()
        print("success")
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Insertion of  Bitcoin News_ Update to Table

In [9]:
#function to insert Bitcoin_News
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_News_Update (id, date, title)
                Values(%s, %s, %s)
                ON CONFLICT (id) DO NOTHING
                ''',(index,row['date'], row['title']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Fetching and Inserting Data

In [13]:
#Main Function to orchestrate the data fetching and insertion
def main():
    # fetch Bitcoin Data
    bitcoin_data = fetch_bitcoin_data("BTC","USD")
    #Scrape Bitcoin News data
    bitcoin_titles_df = get_bitcoin_news()
    #connect to postgresSQL database
    conn = psycopg2.connect(Database_URL)

    if conn:
        #Insert data into database
        insert_bitcoin_data(conn,bitcoin_data)
        insert_bitcoin_news(conn,bitcoin_titles_df)

        #close the database connection
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__== '__main__':
    main()

Python-dotenv could not parse statement starting at line 11


success


## dropping

In [14]:
import os
import psycopg2

def drop_tables(table_names, conn):
    """
    Drops the specified tables from the PostgreSQL database.

    Args:
    table_names (list of str): List of table names to be dropped.
    conn: The psycopg2 connection object to the PostgreSQL database.
    """
    # Create a cursor object
    cur = conn.cursor()

    # Iterate over the list of table names and drop each one
    for table_name in table_names:
        drop_table_query = f'DROP TABLE IF EXISTS {table_name} CASCADE'
        cur.execute(drop_table_query)
        print(f'Table {table_name} dropped.')

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()

if __name__ == "__main__":
    # List of table names to be dropped
    table_names = [
        'bitcoin_scrape3'
        
        
    ]

    # Connect to the PostgreSQL database using the DATABASE_URL environment variable
    DATABASE_URL = os.getenv('DATABASE_URL')
    conn = psycopg2.connect(DATABASE_URL)

    try:
        # Call the function to drop the tables
        drop_tables(table_names, conn)
    finally:
        if conn:
            conn.close()


Table bitcoin_scrape3 dropped.
